### First of all - we need to parse the headers and extract declarations for functions and data types

In [1]:
from cffi import FFI
ffi = FFI()

In [2]:
finalDeclarations=[]

In [3]:
datatypes='''
typedef signed char int8_t;
typedef int int32_t;
typedef unsigned short int uint16_t;
typedef unsigned int uint32_t;
typedef unsigned long int uint64_t;
typedef signed char int_least8_t;
typedef short int int_least16_t;
typedef struct RandomOtherStruct{
    RandomStruct* randomData;
    FfiResult* result;
} RandomOtherStruct;
typedef struct RandomStruct{
    int32_t error_code;
    FfiResult result;
} RandomStruct;
typedef struct FfiResult{
    int32_t error_code;
    char* description;
} FfiResult;
typedef struct Authenticator{
    user_h* core_tx;
    void* _core_joiner;
} Authenticator;
typedef struct Registered{
    int8_t* sym_enc_key;
    int8_t sym_enc_key2;
} Registered;
typedef enum MDataAction {
 MDataAction_Insert,
 MDataAction_Update,
 MDataAction_Delete,
 MDataAction_ManagePermissions,
} MDataAction;
typedef struct UserPermissionSet {
 SignPubKeyHandle user_h;
 FfiPermissionSet perm_set;
} UserPermissionSet;
'''

In [4]:
import re
oneliners=re.compile('typedef.+?;')
myStructTypes=re.compile('typedef struct .+?}.+?;', flags=re.DOTALL)
myEnumTypes=re.compile('typedef enum .+?}.+?;', flags=re.DOTALL)
structAliasName=re.compile('struct .+?{')
enumAliasName=re.compile('enum .+?{')
allOneLiners=oneliners.findall(datatypes)
allMyStructs=myStructTypes.findall(datatypes)
allMyEnums=myEnumTypes.findall(datatypes)

allOneLiners

In [5]:
for oneline in allOneLiners:
    print(oneline)
    ffi.cdef(oneline)
    finalDeclarations.append(oneline)

typedef signed char int8_t;
typedef int int32_t;
typedef unsigned short int uint16_t;
typedef unsigned int uint32_t;
typedef unsigned long int uint64_t;
typedef signed char int_least8_t;
typedef short int int_least16_t;


In [6]:
def getVarProperties(line):
    isPointer = '*' in line
    if isPointer:
        isPointer = 'isPointer'
    else:
        isPointer = 'noPointer'
    name=line.split()[-1][:-1]
    datatype = ' '.join(line.split()[:-1])
    return name, datatype.strip('*'), isPointer

In [7]:
def getAllProperties(struct,dictWithVarsAndTypes={}):
    if 'enum' in struct.splitlines()[0]:
        dictWithVarsAndTypes[struct.splitlines()[-1][2:-1]]={}
        dictWithVarsAndTypes[struct.splitlines()[-1][2:-1]]['declaration']=structAliasName.sub('struct {',struct)
        return struct.splitlines()[-1][2:-1], 'enum', dictWithVarsAndTypes
        #return struct.splitlines()[-1][2:-1], 'enum', enumAliasName.sub('enum {',struct)
    if 'struct' in struct.splitlines()[0]:
        lali=[]
        types=[]
        for i in struct.splitlines()[1:-1]:
            lali.append(getVarProperties(i))
            types.append(lali[-1][1])
        dictWithVarsAndTypes[struct.splitlines()[-1][2:-1]]={}
        dictWithVarsAndTypes[struct.splitlines()[-1][2:-1]]['nameVarTypes']=lali
        dictWithVarsAndTypes[struct.splitlines()[-1][2:-1]]['types']=types
        dictWithVarsAndTypes[struct.splitlines()[-1][2:-1]]['declaration']=structAliasName.sub('struct {',struct)
        return struct.splitlines()[-1][2:-1], 'struct', dictWithVarsAndTypes

getAllProperties(allMyEnums[0])

In [8]:
def checkIfDefined(name,checkDict):
    for onetype in checkDict[name]['types']:
        try:
            ffi.typeof(onetype)
        except:
            return False
    return True

getAllProperties(allMyStructs[3],myDict)

myDict.keys()

checkIfDefined('Authenticator',myDict)

In [9]:
definedStructs=[]
myDict={}

for oneStruct in allMyEnums:
    name, datatype, myDict = getAllProperties(oneStruct,myDict)
    print(myDict[name]['declaration'])
    #print(myDict)
    ffi.cdef(myDict[name]['declaration'])
    finalDeclarations.append(myDict[name]['declaration'])
    definedStructs.append(name)
    

typedef enum MDataAction {
 MDataAction_Insert,
 MDataAction_Update,
 MDataAction_Delete,
 MDataAction_ManagePermissions,
} MDataAction;


In [10]:

for oneStruct in allMyStructs:
    name, datatype, myDict = getAllProperties(oneStruct,myDict)
    if checkIfDefined(name, myDict):
        print(myDict[name]['declaration'])
        #print(myDict)
        ffi.cdef(myDict[name]['declaration'])
        finalDeclarations.append(myDict[name]['declaration'])
        definedStructs.append(name)
    

typedef struct {
    int32_t error_code;
    char* description;
} FfiResult;
typedef struct {
    int8_t* sym_enc_key;
    int8_t sym_enc_key2;
} Registered;


In [11]:
def getNotDefinedStructs(definedStructs,allStructDict):
    notDefined=[]
    for oneName in allStructDict.keys():
        if oneName not in definedStructs:
            notDefined.append(oneName)
    return notDefined

In [12]:
def defineDatatTypes(maxIterations=int(1e3)):
    for i in range(maxIterations):
        #print(i)
        toBeDefined = getNotDefinedStructs(definedStructs,myDict)
        #print(toBeDefined)
        if len(toBeDefined) == 0:
            print(finished)
            break
        for oneDefine in toBeDefined:
            if checkIfDefined(oneDefine, myDict):
                print(myDict[oneDefine]['declaration'])
                #print(myDict)
                ffi.cdef(myDict[oneDefine]['declaration'])
                finalDeclarations.append(myDict[oneDefine]['declaration'])
                definedStructs.append(oneDefine)
                
    return finalDeclarations, toBeDefined

In [13]:
finalDeclarations, stillNotDefined = defineDatatTypes()

typedef struct {
    int32_t error_code;
    FfiResult result;
} RandomStruct;
typedef struct {
    RandomStruct* randomData;
    FfiResult* result;
} RandomOtherStruct;


In [14]:
stillNotDefined

['UserPermissionSet', 'Authenticator']

In [15]:
for eachline in finalDeclarations:
    print(eachline)

typedef signed char int8_t;
typedef int int32_t;
typedef unsigned short int uint16_t;
typedef unsigned int uint32_t;
typedef unsigned long int uint64_t;
typedef signed char int_least8_t;
typedef short int int_least16_t;
typedef enum MDataAction {
 MDataAction_Insert,
 MDataAction_Update,
 MDataAction_Delete,
 MDataAction_ManagePermissions,
} MDataAction;
typedef struct {
    int32_t error_code;
    char* description;
} FfiResult;
typedef struct {
    int8_t* sym_enc_key;
    int8_t sym_enc_key2;
} Registered;
typedef struct {
    int32_t error_code;
    FfiResult result;
} RandomStruct;
typedef struct {
    RandomStruct* randomData;
    FfiResult* result;
} RandomOtherStruct;


myDict['FfiResult']['nameVarTypes']

In [16]:
def checkIfNumeric(dataType):
    return dataType in ' '.join(allOneLiners)

In [17]:
def printDeclarationForChar(name):
    print('''        if type(''' + name + ''') == bytes:
            self.''' + name + ''' = ffi.new('char[]',''' + name + ''')
        else:
            print(\'''' + name + ''' is not data type bytes')
            self.''' + name + ''' = ffi.new('char[]',b'')''')
    return 'self.'+name

In [18]:
def printDeclarationForNumeric(name,dataType,isPointer):
    if 'isPointer' == isPointer:
        print('''        self.''' + name + ''' = ffi.new(\'''' + dataType + ''' *',''' + name + ''')''')
        return 'self.'+name
    else:
        print('''        self.''' + name + ''' = ''' + name + '''''')
        return 'self.'+name

In [19]:
def printDeclarationForStructs(name,dataType,isPointer):

    if 'isPointer' == isPointer:
        print('''        self.''' + name + ''' = ''' + dataType + '''()''')
        return 'self.'+name+'.entity'
    else:
        print('''        self.''' + name + ''' = ''' + dataType + '''()''')
        return 'self.'+name+'.entity[0]'

allOneLiners

checkIfNumeric('uint16_t')

In [20]:
for name in myDict:
    if 'nameVarTypes' in myDict[name].keys():
        print('class ' + name + ':')
        initLine='    def __init__(self'
        for oneDef in myDict[name]['nameVarTypes']:
            if 'char' in oneDef[1]:
                initLine += ',' + oneDef[0] + '=b\'\''
            elif checkIfNumeric(oneDef[1]):
                initLine += ',' + oneDef[0] + '=0'
        initLine += '):'
        print(initLine)

        innerVars=[]
        for oneDef in myDict[name]['nameVarTypes']:
            if 'char' in oneDef[1]:
                innerVars.append(printDeclarationForChar(oneDef[0]))
            elif checkIfNumeric(oneDef[1]):
                innerVars.append(printDeclarationForNumeric(oneDef[0],oneDef[1],oneDef[2]))
            else:
                innerVars.append(printDeclarationForStructs(oneDef[0],oneDef[1],oneDef[2]))

        print()
        finalLine='''        self.entity = ffi.new(\'''' + name + ''' *\',['''
        for innerVar in innerVars:
            finalLine += innerVar + ','
        finalLine = finalLine[:-1] + '])'
        print(finalLine)
        print()

class FfiResult:
    def __init__(self,error_code=0,description=b''):
        self.error_code = error_code
        if type(description) == bytes:
            self.description = ffi.new('char[]',description)
        else:
            print('description is not data type bytes')
            self.description = ffi.new('char[]',b'')

        self.entity = ffi.new('FfiResult *',[self.error_code,self.description])

class UserPermissionSet:
    def __init__(self):
        self.user_h = SignPubKeyHandle()
        self.perm_set = FfiPermissionSet()

        self.entity = ffi.new('UserPermissionSet *',[self.user_h.entity[0],self.perm_set.entity[0]])

class Registered:
    def __init__(self,sym_enc_key=0,sym_enc_key2=0):
        self.sym_enc_key = ffi.new('int8_t *',sym_enc_key)
        self.sym_enc_key2 = sym_enc_key2

        self.entity = ffi.new('Registered *',[self.sym_enc_key,self.sym_enc_key2])

class RandomOtherStruct:
    def __init__(self):
        self.randomData = RandomStruct()
      

In [21]:
class cstr:
    def __init__(self,myStringToCreate):
        self.entity = ffi.new('char[]',myStringToCreate)

In [22]:
def to_void(data):
    return ffi.new_handle(data)

In [23]:
def from_void(data):
    return ffi.from_handle(data)